# Churn prediction 25/26 - Project for Data Science and AI for Business
Authors: Andreea Patarlageanu and Martin Lau

In [1]:
# We will put all imports here
import pandas as pd
import numpy as np


**EDITOR NOTES**

- changed structure of the document a bit. do EDA first, then start changing columns or mappings with feature engineering.
- Added more to feature engineering part

## 1. Exploratory Data Analysis

We will start with an exploratory data analysis in order to better understand the data and format it efficiently for later.

### Loading data, checking features

#### Load data

In [80]:
# ! Add code to verify if the file exists!
df = pd.read_parquet("Data/train.parquet")

In [81]:
df.shape

(17499636, 19)

In [82]:
df.head(5)

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


#### Understanding each feature.

In this section, we break down each column to get a full understanding of what each information our dataset can give us. 

Some features are self explanatory:
- `userId`: Internal user ID given to unique accounts. Seems arbitrarily determined.
- `sessionId`: Internal session ID given to unique sessions. Seems arbitrarily determined.
- `ts`: Linux time stamp.
- `gender`: Male or female (we shall map 0 or 1 to each gender later).
- `location`: Geographical location
- `registration`: Time stamp for when the user ID has registered onto the platform.
- `song`, `artist`, `time`: What song/artist they played in that session, at what time. 
- `length`: Length of the song played (According to Google, *Seven Nation Army* is a 3 minute 52 second song, or 232 seconds).
- `firstName`, `lastName`: It doesn't take a genius to figure this one out.

The `status` column corresponds to HTTP codes. (200: OK, 307: Temporary redirect, 404: Not found)

In [84]:
print(df['status'].unique())

[200 307 404]


The `level` column corresponds to different subscription plans the user has signed up to.

In [85]:
print(df['level'].unique())

['paid' 'free']


The `page` column corresponds to the page the user's session is currently in.

In [86]:
print(df['page'].unique())

['NextSong' 'Downgrade' 'Help' 'Home' 'Thumbs Up' 'Add Friend'
 'Thumbs Down' 'Add to Playlist' 'Logout' 'About' 'Settings'
 'Save Settings' 'Cancel' 'Cancellation Confirmation' 'Submit Downgrade'
 'Roll Advert' 'Upgrade' 'Error' 'Submit Upgrade']


The `userAgent` corresponds to the platform the user is logging onto the platform with.

In [89]:
print(f"Number of unique user Agents: {len(df['userAgent'].unique())}")
print(df['userAgent'].unique()[:5])

85
['"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0'
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'
 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0']


The `method` corresponds to HTTP methods. PUT = update or replace existing element, GET = retrieve HTTP element.   

In [90]:
print(df['method'].unique())

['PUT' 'GET']


#### Inspecting the test dataset.

It is important we check the test dataset to see if there are any large differences between our train and test sets.

In [83]:
df_test = pd.read_parquet("Data/test.parquet")

In [92]:
test_pages = set(df_test['page'].dropna().unique())
train_pages = set(df['page'].dropna().unique())

only_in_test = test_pages - train_pages
print("Pages in test but NOT in train:")
print(only_in_test)

only_in_train = train_pages - test_pages
print("\nPages in train but NOT in test:")
print(only_in_train)

Pages in test but NOT in train:
{'Submit Registration', 'Login', 'Register'}

Pages in train but NOT in test:
{'Cancel', 'Cancellation Confirmation'}


In [69]:
print(df_test['auth'].unique())
print(df['auth'].unique())

['Logged In' 'Logged Out' 'Guest']
['Logged In' 'Cancelled']


**2 key observations from the test dataset vs the train dataset:**

1. In the train, we observe that there is a cancellation confirmation and cancel in the page column. We infer that this should be how we determine whether a user has churned or not. In the test, there are people signing up for the service, and not the other way around.
2. Different values in the auth column. The test set has `Logged Out` and `Guest`, while all users in the train set are logged in.

##### Checking the Guest and LoggedOut sessions.

In [95]:
guest_rows = df_test[df_test['auth'] == 'Guest']
guest_rows.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
63324,404,None,None,free,None,1261737,1538377625000,Guest,Error,15008,None,0,None,GET,NaN,None,None,2018-10-01 07:07:05,2018-10-01 00:00:08
64030,200,None,None,free,None,1261737,1538377871000,Guest,Home,15008,None,1,None,GET,NaN,None,None,2018-10-01 07:11:11,2018-10-01 00:00:08
64320,200,None,None,free,None,1261737,1538377969000,Guest,About,15008,None,2,None,GET,NaN,None,None,2018-10-01 07:12:49,2018-10-01 00:00:08
64518,200,None,None,free,None,1261737,1538378050000,Guest,Help,15008,None,3,None,GET,NaN,None,None,2018-10-01 07:14:10,2018-10-01 00:00:08
65045,200,None,None,free,None,1261737,1538378238000,Guest,Home,15008,None,4,None,GET,NaN,None,None,2018-10-01 07:17:18,2018-10-01 00:00:08


Looks like people looking around at information on the music service. 

In [91]:
logged_out = df_test[df_test['auth'] == 'Logged Out']
logged_out.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
17,200,None,None,paid,None,1261737,1538352008000,Logged Out,Home,8615,None,87,None,GET,NaN,None,None,2018-10-01 00:00:08,2018-10-01 00:00:08
71,307,None,None,free,None,1261737,1538352041000,Logged Out,Login,7433,None,0,None,PUT,NaN,None,None,2018-10-01 00:00:41,2018-10-01 00:00:08
354,200,None,None,free,None,1261737,1538352182000,Logged Out,Home,25003,None,4,None,GET,NaN,None,None,2018-10-01 00:03:02,2018-10-01 00:00:08
472,200,None,None,free,None,1261737,1538352254000,Logged Out,Home,9930,None,2,None,GET,NaN,None,None,2018-10-01 00:04:14,2018-10-01 00:00:08
476,307,None,None,free,None,1261737,1538352255000,Logged Out,Login,9930,None,3,None,PUT,NaN,None,None,2018-10-01 00:04:15,2018-10-01 00:00:08


#### Inspecting the submission file structure

It may also be helpful to see what the example submission looks like, to see what is expected.

In [66]:
target = pd.read_csv("Data/example_submission.csv")
print(f"Length of submission example: {len(target)} rows.")

Length of submission example: 2904 rows.


In [51]:
target.head(10)

,id,target
0,1128274,0
1,1782451,1
2,1611542,1
3,1241663,0
4,1653104,1
5,1486416,1
6,1442381,1
7,1117734,1
8,1212570,1
9,1755776,1


This lets us see what we need to predict.

## 2. Feature engineering

### Preliminary column transformations

Now, let's inspect a bit more the content of the columns that we think it would make sense to change the type:

In [96]:
print(df.columns)
print(df.dtypes)

Index(['status', 'gender', 'firstName', 'level', 'lastName', 'userId', 'ts',
       'auth', 'page', 'sessionId', 'location', 'itemInSession', 'userAgent',
       'method', 'length', 'song', 'artist', 'time', 'registration'],
      dtype='object')
status                    int64
gender                   object
firstName                object
level                    object
lastName                 object
userId                   object
ts                        int64
auth                     object
page                     object
sessionId                 int64
location                 object
itemInSession             int64
userAgent                object
method                   object
length                  float64
song                     object
artist                   object
time             datetime64[us]
registration     datetime64[us]
dtype: object


We see that the order of the columns is not good. Moreover, some types can be changed for a better model comprehension.

First, let's reorder the columns like following:
- *User identifiers*
- *session information*
- *timestamps*
- *user context*
- *action data*
- *content listened to*

In [8]:
print(df['status'].unique())
print(df['level'].unique())
print(df['page'].unique())
print(df['method'].unique())
print(df['auth'].unique())
print(df['page'].unique())

[200 307 404]
['paid' 'free']
['NextSong' 'Downgrade' 'Help' 'Home' 'Thumbs Up' 'Add Friend'
 'Thumbs Down' 'Add to Playlist' 'Logout' 'About' 'Settings'
 'Save Settings' 'Cancel' 'Cancellation Confirmation' 'Submit Downgrade'
 'Roll Advert' 'Upgrade' 'Error' 'Submit Upgrade']
['PUT' 'GET']
['Logged In' 'Cancelled']
['NextSong' 'Downgrade' 'Help' 'Home' 'Thumbs Up' 'Add Friend'
 'Thumbs Down' 'Add to Playlist' 'Logout' 'About' 'Settings'
 'Save Settings' 'Cancel' 'Cancellation Confirmation' 'Submit Downgrade'
 'Roll Advert' 'Upgrade' 'Error' 'Submit Upgrade']


In [6]:
order_columns = [
    'userId',
    'firstName', 
    'lastName',
    'gender',
    'registration',
    
    'sessionId',
    'itemInSession',
    
    'ts',
    'time',
    
    'level',
    'auth',
    'location',
    'userAgent',
    
    'page',
    'method',
    'status',
    
    'song',
    'artist',
    'length'
]

In [7]:
df = df[order_columns]

Now, let's change some of the columns in the proper types:

- gender should be binary: 0 for 'F' and 1 for 'M'
- the level, meaning the level of the subscription, can be free or paid so we set: 0 for 'free' and 1 for 'paid'
- the page action shows the actions of the users/pages visited. Since there are many categories, we will leave that column's type to be 'object'
- the method (HTTP request methods) we set to be 0 for GET (retrieving/viewing data - reading) and 1 for PUT (sending/updating data - writing, like saving a song)
- for 'auth', let's set 0 for 'Cancelled' and 1 for 'Logged In'

In [97]:
df["gender"] = df["gender"].map({'F':0, 'M':1})
df["level"] = df["level"].map({'free' : 0, 'paid': 1})
df["method"] = df["method"].map({'GET' : 0, 'PUT' : 1 })

# ! THIS WORKS ONLY ON TRAIN, NOT ON TEST !
df["auth"] = df["auth"].map({'Cancelled' : 0, 'Logged In' : 1 }) 

Moreover, we see that 'userId''s type is object. Let's see if there are values in this column which cannot be converted to int:

In [98]:
try:
    df['userId'].astype(int)
    df['userId'] = df['userId'].astype(int)
    print("Only integers in userId - transformation successful")
except Exception as e:
    print(f"Some non integer values in userId: {e}")

Only integers in userId - transformation successful


Finally, we create the target columns based on churn.

In [99]:
churned_users = df[df['page'] == 'Cancellation Confirmation']['userId'].unique()
df['churned'] = df['userId'].isin(churned_users).astype(int)

Let's go again through the types of the columns:

In [100]:
df.dtypes

status                    int64
gender                    int64
firstName                object
level                     int64
lastName                 object
userId                    int64
ts                        int64
auth                      int64
page                     object
sessionId                 int64
location                 object
itemInSession             int64
userAgent                object
method                    int64
length                  float64
song                     object
artist                   object
time             datetime64[us]
registration     datetime64[us]
churned                   int64
dtype: object

In [101]:
df.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration,churned
0,200,1,Shlok,1,Johnson,1749042,1538352001000,1,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21,1
992,200,1,Shlok,1,Johnson,1749042,1538352525000,1,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21,1
1360,200,1,Shlok,1,Johnson,1749042,1538352703000,1,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21,1
1825,200,1,Shlok,1,Johnson,1749042,1538352935000,1,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21,1
2366,200,1,Shlok,1,Johnson,1749042,1538353200000,1,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21,1


### Missing values

In [41]:
df.isnull().sum()

userId                  0
firstName               0
lastName                0
gender           17499636
registration            0
sessionId               0
itemInSession           0
ts                      0
time                    0
level            17499636
auth             17499636
location                0
userAgent               0
page                    0
method           17499636
status                  0
song              3208203
artist            3208203
length            3208203
dtype: int64

We see that the columns 'song', 'artist' and 'length' have the same number of missing values. This might indicate the possibility of having the missing values in the same lines for all 3 columns. Let's check that:

In [42]:
song_missing = df['song'].isnull()
artist_missing = df['artist'].isnull()
length_missing = df['length'].isnull()

# Let's check if they all occur in the same rows:
print("Are all missing in the same rows? Answer: ", (song_missing == artist_missing).all() and (artist_missing == length_missing).all() )

Are all missing in the same rows? Answer:  True


So we see that all the missing values of the 3 columns appear in the same lines. Let's check what percentage of the data all these lines represent:

In [43]:
rows_missing_values = song_missing.sum()
percentage = rows_missing_values / df.shape[0] * 100
print(f"Percentage of rows with missing values: {percentage:.2f}%" )

Percentage of rows with missing values: 18.33%


#TODO: should we delete or keep??????????????????

Now, we will just make sure that the values in 'registration' and 'time' make sense (not too long ago, or inthe future):

In [44]:
print("Range of dates in the registration column:")
print(f"Earliest registration: {df['registration'].min()}")
print(f"Latest registration: {df['registration'].max()}")

print()

print("Range of dates in the time column:")
print(f"Earliest time: {df['time'].min()}")
print(f"Latest time: {df['time'].max()}")

Range of dates in the registration column:
Earliest registration: 2017-10-14 22:05:25
Latest registration: 2018-11-19 23:34:34

Range of dates in the time column:
Earliest time: 2018-10-01 00:00:01
Latest time: 2018-11-20 00:00:00


We see that the earliest time was almost one year after the earliest registration.

#TODO: what do we do in this case?????????????????????????????????????

### Row aggregation by userID

We remove the following columns as we deem that they will not influence whether a user will churn or not.
- `firstName` and `lastName` (independence from churn)
- `sessionId` (arbitrary, independent from churn)

Since the task is to predict whether a user will churn, given a user ID and other session data, it makes the most sense to aggregate all rows by userID. For each user ID, we can then fill out different information:
- `userID` 
- `gender`
- `n_sessions`
- `avg_session_length`
- `location`
- `registration`
- `n_songs`
- `n_artists`
- `churn` (target)

In [109]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['registration'] = pd.to_datetime(df['registration'])

df['session_length'] = df.groupby(['userId', 'sessionId'])['ts'].transform(lambda x: x.max() - x.min())
df['song_played'] = df['page'] == 'NextSong'

user_df = df.groupby('userId').agg({
    'gender': 'first',
    'registration': 'first',
    'location': 'last',
    'level': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'sessionId': 'nunique',  # number of sessions
    'itemInSession': 'max',
    'ts': ['min', 'max'],
    'session_length': 'mean',
    'song_played': 'sum',
    'artist': pd.Series.nunique,
    'length': 'sum',
    'churned': 'max'    
}).reset_index()

user_df.columns = ['userId', 'gender', 'registration', 'location', 'level',
                   'num_sessions', 'max_item_in_session', 'ts_min', 'ts_max', 'avg_session_length',
                   'num_songs_played', 'unique_artists', 'total_length', 'churned']

user_df['days_active'] = (user_df['ts_max'] - user_df['ts_min']).dt.days
user_df['membership_length'] = (user_df['ts_max'] - user_df['registration']).dt.days

In [110]:
user_df.head(10)

,userId,gender,registration,location,level,num_sessions,max_item_in_session,ts_min,ts_max,avg_session_length,num_songs_played,unique_artists,total_length,churned,days_active,membership_length
0,1000025,1,2018-07-10 09:30:08,"New Haven-Milford, CT",1,17,486,2018-10-02 08:59:29,2018-10-18 20:33:05,0 days 13:42:02.882294264,1662,1162,417296.59169,1,16,100
1,1000035,0,2018-09-12 19:28:22,"Columbia, SC",1,21,228,2018-10-05 18:29:46,2018-11-15 03:53:11,0 days 07:31:30.928020565,1266,916,310364.86590,0,40,63
2,1000083,1,2018-09-07 18:01:49,"Cincinnati, OH-KY-IN",1,11,171,2018-10-01 05:37:45,2018-10-12 10:04:58,0 days 06:07:04.669463087,501,427,122606.27093,1,11,34
3,1000103,0,2018-09-22 07:27:25,"Cleveland-Elyria, OH",0,3,53,2018-10-04 17:31:17,2018-11-08 18:28:40,0 days 02:02:59.360000,57,56,13554.73009,0,35,47
4,1000164,0,2018-08-12 09:32:01,"Phoenix-Mesa-Scottsdale, AZ",1,15,215,2018-10-01 17:40:18,2018-11-19 13:04:25,0 days 08:36:20.251945525,847,660,209060.65753,0,48,99
5,1000168,1,2018-08-08 16:06:13,"Tallahassee, FL",1,8,203,2018-10-04 14:23:24,2018-11-14 10:38:56,0 days 07:45:32.364035087,545,451,135277.67608,0,40,97
6,1000182,0,2018-07-03 12:06:05,"Chicago-Naperville-Elgin, IL-IN-WI",1,4,216,2018-10-19 22:47:56,2018-11-09 23:50:22,0 days 07:52:31.256351039,357,317,87856.68704,0,21,129
7,1000194,0,2018-09-10 16:51:26,"Palestine, TX",0,3,61,2018-10-22 19:50:07,2018-11-17 22:18:07,0 days 02:32:09.419354838,75,73,17678.12439,0,26,68
8,1000214,1,2018-09-18 00:37:57,"Worcester, MA-CT",1,20,391,2018-10-03 11:01:03,2018-11-18 06:56:10,0 days 12:32:15.061309523,1364,993,341651.54807,0,45,61
9,1000233,1,2018-09-28 15:45:39,"New York-Newark-Jersey City, NY-NJ-PA",1,2,56,2018-10-22 18:28:39,2018-11-16 01:05:35,0 days 02:55:00,51,49,12680.28384,0,24,48


## 3. Preprocessing